# Inception Score

In [1]:

import os, sys
import numpy as np
from PIL import Image
import glob
import tensorflow as tf

def get_images(filename): 
    img = Image.open(filename)
    img.load()
    img = img.convert("RGB")
    data = np.asarray( img, dtype="int32")
    data = np.moveaxis(data, -1, 0)
    return data

filenames = glob.glob(os.path.join('my_cgan_output', '*.png'))  # Change to generated image folder
images = [get_images(filename) for filename in filenames]
images = np.stack(images, axis=0)

print(images.shape)
print(len(images))
print(get_inception_score(images))

(501, 3, 480, 640)
501


NameError: name 'get_inception_score' is not defined

In [1]:
# Following Code from: https://github.com/tsc2017/Inception-Score
import tensorflow as tf
import os, sys
import functools
import numpy as np
import time

import imageio
from PIL import Image

from tensorflow.python.ops import array_ops
from tensorflow.python.ops import functional_ops
tfgan = tf.contrib.gan

session = tf.InteractiveSession()

# A smaller BATCH_SIZE reduces GPU memory usage, but at the cost of a slight slowdown
BATCH_SIZE = 10

# Run images through Inception.
inception_images = tf.placeholder(tf.float32, [BATCH_SIZE, 3, None, None])
def inception_logits(images = inception_images, num_splits = 1):
    images = tf.transpose(images, [0, 2, 3, 1])
    size = 299
    images = tf.image.resize_bilinear(images, [size, size])
    generated_images_list = array_ops.split(images, num_or_size_splits = num_splits)
    logits = functional_ops.map_fn(
        fn = functools.partial(tfgan.eval.run_inception, output_tensor = 'logits:0'),
        elems = array_ops.stack(generated_images_list),
        parallel_iterations = 1,
        back_prop = False,
        swap_memory = True,
        name = 'RunClassifier')
    logits = array_ops.concat(array_ops.unstack(logits), 0)
    return logits

logits=inception_logits()

def get_inception_probs(inps):
    n_batches = len(inps) // BATCH_SIZE
    preds = np.zeros([n_batches * BATCH_SIZE, 1000], dtype = np.float32)
    for i in range(n_batches):
        inp = inps[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] / 255. * 2 - 1
        preds[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] = logits.eval({inception_images:inp})[:, :1000]
    preds = np.exp(preds) / np.sum(np.exp(preds), 1, keepdims=True)
    return preds

def preds2score(preds, splits=10):
    scores = []
    for i in range(splits):
        part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
        kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

def get_inception_score(images, splits=10):
    assert(type(images) == np.ndarray)
    assert(len(images.shape) == 4)
    assert(images.shape[1] == 3)
    assert(np.min(images[0]) >= 0 and np.max(images[0]) > 10), 'Image values should be in the range [0, 255]'

    print('Calculating Inception Score with %i images in %i splits' % (images.shape[0], splits))
    start_time=time.time()
    preds = get_inception_probs(images)
    mean, std = preds2score(preds, splits)
    print('Inception Score calculation time: %f s' % (time.time() - start_time))
    return mean, std  # Reference values: 11.34 for 49984 CIFAR-10 training set images, or mean=11.31, std=0.08 if in 10 splits.


    


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


2021-07-07 17:34:20.636009: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-07-07 17:34:20.655877: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2599990000 Hz
2021-07-07 17:34:20.656303: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5567c5e402a0 executing computations on platform Host. Devices:
2021-07-07 17:34:20.656321: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


>> Downloading http://download.tensorflow.org/models/frozen_inception_v1_2015_12_05.tar.gz 22.7%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



>> Downloading http://download.tensorflow.org/models/frozen_inception_v1_2015_12_05.tar.gz 35.0%

# FID

In [1]:
# !mkdir orig_album

In [2]:
import os, sys
import numpy as np
from PIL import Image
import glob
import tensorflow as tf

def get_images(filename):
    
    img = Image.open(filename)
    img.load()
    img = img.convert("RGB")
    img = img.resize((28,28))
    data = np.asarray( img, dtype="int32")
    
    data = np.moveaxis(data, -1, 0)
    
    return data

filenames = glob.glob(os.path.join('my_cgan_output', '*.png'))  # Change to generated image folder
images = [get_images(filename) for filename in filenames]
images = np.stack(images, axis=0)

print(images.shape)
print(len(images))

filenames = glob.glob(os.path.join('output', '*.jpg*'))  # Change to real image folder
orig_images = [get_images(filename) for filename in filenames]
orig_images = np.stack(orig_images, axis=0)
print(orig_images.shape)
print(len(orig_images))

print(get_fid(images,orig_images))


(501, 3, 28, 28)
501


ValueError: need at least one array to stack

In [ ]:
# Code from https://github.com/tsc2017/Frechet-Inception-Distance
import tensorflow as tf
import os, sys
import functools
import numpy as np
import time
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import functional_ops
tfgan = tf.contrib.gan

session = tf.InteractiveSession()

# A smaller BATCH_SIZE reduces GPU memory usage, but at the cost of a slight slowdown
BATCH_SIZE = 10

# Run images through Inception.
inception_images = tf.placeholder(tf.float32, [BATCH_SIZE, 3, None, None])
activations1 = tf.placeholder(tf.float32, [None, None], name = 'activations1')
activations2 = tf.placeholder(tf.float32, [None, None], name = 'activations2')
fcd = tfgan.eval.frechet_classifier_distance_from_activations(activations1, activations2)

def inception_activations(images = inception_images, num_splits = 1):
    images = tf.transpose(images, [0, 2, 3, 1])
    size = 299
    images = tf.image.resize_bilinear(images, [size, size])
    generated_images_list = array_ops.split(images, num_or_size_splits = num_splits)
    activations = functional_ops.map_fn(
        fn = functools.partial(tfgan.eval.run_inception, output_tensor = 'pool_3:0'),
        elems = array_ops.stack(generated_images_list),
        parallel_iterations = 1,
        back_prop = False,
        swap_memory = True,
        name = 'RunClassifier')
    activations = array_ops.concat(array_ops.unstack(activations), 0)
    return activations

activations =inception_activations()

def get_inception_activations(inps):
    n_batches = inps.shape[0]//BATCH_SIZE
    act = np.zeros([n_batches * BATCH_SIZE, 2048], dtype = np.float32)
    for i in range(n_batches):
        inp = inps[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] / 255. * 2 - 1
        act[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] = activations.eval(feed_dict = {inception_images: inp})
    return act

def activations2distance(act1, act2):
     return fcd.eval(feed_dict = {activations1: act1, activations2: act2})
        
def get_fid(images1, images2):
    assert(type(images1) == np.ndarray)
    assert(len(images1.shape) == 4)
    assert(images1.shape[1] == 3)
    assert(np.min(images1[0]) >= 0 and np.max(images1[0]) > 10), 'Image values should be in the range [0, 255]'
    assert(type(images2) == np.ndarray)
    assert(len(images2.shape) == 4)
    assert(images2.shape[1] == 3)
    assert(np.min(images2[0]) >= 0 and np.max(images2[0]) > 10), 'Image values should be in the range [0, 255]'
    assert(images1.shape == images2.shape), 'The two numpy arrays must have the same shape'
    print('Calculating FID with %i images from each distribution' % (images1.shape[0]))
    start_time = time.time()
    act1 = get_inception_activations(images1)
    act2 = get_inception_activations(images2)
    fid = activations2distance(act1, act2)
    print('FID calculation time: %f s' % (time.time() - start_time))
    return fid

>> Downloading http://download.tensorflow.org/models/frozen_inception_v1_2015_12_05.tar.gz 100.0%